In [13]:
%cd /root/repositories/Wav2Lip-CodeFormer
%rm -rf ./images
%mkdir images

%cd /root/repositories/Wav2Lip-CodeFormer/CodeFormer
%rm -rf ./results
%mkdir results 

/root/repositories/Wav2Lip-CodeFormer


/root/repositories/Wav2Lip-CodeFormer/CodeFormer


In [14]:
%cd /root/repositories/Wav2Lip-CodeFormer/Wav2Lip 
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '../input/driving.mp4' --audio '../input/generated.wav'

/root/repositories/Wav2Lip-CodeFormer/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 169
/root/repositories/Wav2Lip-CodeFormer/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 451)
Length of mel chunks: 165
100%|███████████████████████████████████████████| 11/11 [00:04<00:00,  2.33it/s]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100%|█████████████████████████████████████████████| 2/2 [00:10<00:00,  5.16s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-

In [15]:
%cd /root/repositories/Wav2Lip-CodeFormer/Wav2Lip

import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("/root/repositories/Wav2Lip-CodeFormer/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/root/repositories/Wav2Lip-CodeFormer/images', str(frameNumber).zfill(4)+'.jpg'), image)

/root/repositories/Wav2Lip-CodeFormer/Wav2Lip
FPS:  29.916666666666668 Frames:  165


  0%|          | 0/165 [00:00<?, ?it/s]

100%|██████████| 165/165 [00:00<00:00, 1216.12it/s]


In [16]:
%cd /root/repositories/Wav2Lip-CodeFormer/CodeFormer

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/root/repositories/Wav2Lip-CodeFormer/CodeFormer


In [17]:
%cd /root/repositories/Wav2Lip-CodeFormer/CodeFormer

CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /root/repositories/Wav2Lip-CodeFormer/images --bg_upsampler realesrgan

/root/repositories/Wav2Lip-CodeFormer/CodeFormer


Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/165] Processing: 0000.jpg
	detect 1 faces
[2/165] Processing: 0001.jpg
	detect 1 faces
[3/165] Processing: 0002.jpg
	detect 1 faces
[4/165] Processing: 0003.jpg
	detect 1 faces
[5/165] Processing: 0004.jpg
	detect 1 faces
[6/165] Processing: 0005.jpg
	detect 1 faces
[7/165] Processing: 0006.jpg
	detect 1 faces
[8/165] Processing: 0007.jpg
	detect 1 faces
[9/165] Processing: 0008.jpg
	detect 1 faces
[10/165] Processing: 0009.jpg
	detect 1 faces
[11/165] Processing: 0010.jpg
	detect 1 faces
[12/165] Processing: 0011.jpg
	detect 1 faces
[13/165] Processing: 0012.jpg
	detect 1 faces
[14/165] Processing: 0013.jpg
	detect 1 faces
[15/165] Processing: 0014.jpg
	detect 1 faces
[16/165] Processing: 0015.jpg
	detect 1 faces
[17/165] Processing: 0016.jpg
	detect 1 faces
[18/165] Processing: 0017.jpg
	detect 1 faces
[19/165] Processing: 0018.jpg
	detect 1 faces
[20/165] Processing: 0019.jpg
	detect 1 f

In [18]:
%cd /root/repositories/Wav2Lip-CodeFormer/output
%rm ./final.mp4
%rm ./batch_0000.avi

/root/repositories/Wav2Lip-CodeFormer/output
rm: cannot remove './final.mp4': No such file or directory


In [19]:
import os
import ffmpeg

restoredFramesPath = '/root/repositories/Wav2Lip-CodeFormer/CodeFormer/results/images_0.7' + '/final_results/'
processedVideoOutputPath = '/root/repositories/Wav2Lip-CodeFormer/output'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

audio = ffmpeg.input(f'/root/repositories/Wav2Lip-CodeFormer/input/generated.wav')
video = ffmpeg.input(f'/root/repositories/Wav2Lip-CodeFormer/output/batch_0000.avi')
out = ffmpeg.output(video, audio, f'/root/repositories/Wav2Lip-CodeFormer/output/final.mp4')
out.run()  

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600


100%|██████████| 1/1 [00:03<00:00,  3.44s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvid

(None, None)